In [1]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

bootleg


In [2]:
from pathlib import Path
import ujson, os
from tqdm.auto import tqdm
from bootleg.symbols.entity_profile import EntityProfile

In [4]:
# MODIFY THE PATH TO THE DOWNLOADED ENTITY_DB DATA.
entity_profile_cache = Path("/opt/data/cchang/bootleg_train/data/entity_db")
# Print out directory structure
for fold in entity_profile_cache.iterdir():
    # Skip showing our prep directory as that's used when loading a model
    if fold.name in ["prep"]:
        continue
    print(fold.name)
    for sub_file in fold.iterdir():
        print("   ", sub_file.name)
        if sub_file.is_dir():
            for subsub_file in sub_file.iterdir():
                print("       ", subsub_file.name)

kg_mappings
    config.json
    qid2relations
        max_value.json
        value_vocabulary_trie.marisa
        key_vocabulary_trie.marisa
        record_trie.marisa
entity_mappings
    config.json
    alias2id
        config.json
        itoexti.npy
        vocabulary_trie.marisa
    qid2desc.json
    qid2eid
        config.json
        itoexti.npy
        vocabulary_trie.marisa
    alias2qids
        max_value.json
        vocabulary_trie.marisa
        record_trie.marisa
    qid2title.json
type_mappings
    wiki
        config.json
        qid2typenames


In [13]:
from bootleg.utils.utils import load_yaml_file
from bootleg.end2end.bootleg_annotator import BootlegAnnotator
from pathlib import Path

# root_dir = FILL IN FULL PATH TO DIRECTORY WHERE DATA IS DOWNLOADED (i.e., root_dir/data and root_dir/models)
root_dir = Path("/opt/data/cchang/bootleg_train")
# entity_dir = FILL IN PATH TO ENTITY_DB DATA (i.e., tutorial_data/data
data_dir = root_dir / "data"
entity_dir = data_dir / "entity_db"
# model_dir = FILL IN PATH TO MODELS
model_dir = root_dir / "models"


config_in_path = model_dir / "run_config.yaml"

config_args = load_yaml_file(config_in_path)

# set the model checkpoint path
config_args["emmental"]["model_path"] = str(
    model_dir / "last_model.pth"
)

device = -1 # 0 :gpu
# set the path for the entity db and candidate map
config_args["data_config"]["entity_dir"] = str(entity_dir)

ann = BootlegAnnotator(
    config=config_args, device=device, return_embs=False, verbose=True, 
    cache_dir = "/opt/data/cchang/cache"
)

[2022-05-11 10:58:31,456][INFO] emmental.meta:122 - Setting logging directory to: /opt/data/cchang/bootleg_train/logs/2022_05_11/10_58_31/3de169cb
[2022-05-11 10:58:31,475][INFO] emmental.meta:64 - Loading Emmental default config from /home/cchang/miniconda/envs/bootleg/lib/python3.8/site-packages/emmental/emmental-default-config.yaml.
[2022-05-11 10:58:31,475][INFO] emmental.meta:174 - Updating Emmental config from user provided config.
[2022-05-11 10:58:31,476][INFO] emmental.utils.seed:23 - Set random seed to 1234.
[2022-05-11 10:58:31,477][DEBUG] bootleg.end2end.bootleg_annotator:236 - Reading entity database
[2022-05-11 10:58:34,640][DEBUG] bootleg.end2end.bootleg_annotator:251 - Reading entity type database
[2022-05-11 10:58:34,642][DEBUG] bootleg.end2end.bootleg_annotator:262 - Reading entity kg database
[2022-05-11 10:58:34,643][DEBUG] bootleg.end2end.bootleg_annotator:269 - Reading word tokenizers
[2022-05-11 10:58:34,646][DEBUG] urllib3.connectionpool:1001 - Starting new HTTP

In [15]:
sentence = "我很喜欢梵高的画, 露营的时候常常想起他"
res = ann.label_mentions(sentence)
print(res)

Prepping data:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating model:   0%|          | 0/1 [00:00<?, ?it/s]

{'qids': [['Q5582', 'Q11629']], 'probs': [[0.9999488592147827, 0.9997074007987976]], 'titles': [['文森特·梵高', '绘画']], 'cands': [[['NC', 'Q5582', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1'], ['NC', 'Q11629', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1']]], 'cand_probs': [[[5.1136601541657e-05, 0.9999488592147827, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0002925781300291419, 0.9997074007987976, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]], 'char_spans': [[[4, 6], [7, 8]]], 'aliases': [['梵高', '画']]}


In [5]:
import time

# edit_mode=True 需要占用内存 30-40GB

st = time.time()
# Load up ALL profile data - don't forget to set edit_mode = True if you want edit the profile
# As edit_mode triggers the profile to build some index structures for fast editing,
# the loading takes a few minutes for all of wiki
ep = EntityProfile.load_from_cache(entity_profile_cache, edit_mode=False, verbose=True)
print(f"Loaded full ep in {time.time() - st}")
st = time.time()

Loading Entity Symbols
Loading Type Symbols from /opt/data/cchang/bootleg_train/data/entity_db/type_mappings/wiki
Loading KG Symbols
Loaded full ep in 3.8229048252105713


In [28]:
for qid in res['qids'][0]:
    print(qid)
    try:
        #print(ep.get_mentions(qid))
        print(ep.get_title(qid))
        print(ep.get_types(qid,'wiki'))
    except:
        pass

Q5582
文森特·梵高
['人类', '画家', '画家']
Q11629
绘画
['活动', '方法', '风格']


In [29]:
object_methods = [
    method_name for method_name in dir(ep) if callable(getattr(ep, method_name))
]

print(object_methods)

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_read_profile_file', 'add_entity', 'add_mention', 'add_relation', 'add_type', 'get_all_mentions', 'get_all_qids', 'get_all_types', 'get_all_typesystems', 'get_desc', 'get_eid', 'get_entities_of_type', 'get_mentions', 'get_mentions_with_scores', 'get_qid_cands', 'get_qid_count_cands', 'get_relations_between', 'get_relations_tails_for_qid', 'get_title', 'get_type_typeid', 'get_types', 'load_from_cache', 'load_from_jsonl', 'mention_exists', 'prune_to_entities', 'qid_exists', 'reidentify_entity', 'remove_mention', 'remove_relation', 'remove_type', 'save', 'save_to_jsonl', 'update_entity']


In [33]:
# Get the title of an entity
#print("Title:", ep.get_mentions_with_scores("Q11172960"))


In [20]:
qid = "Q5582"
type_system = "wiki"
print("Existing Types:", ep.get_types(qid, type_system))

Existing Types: ['人类', '画家', '画家']


In [31]:
len(set(ep.get_all_types('wiki')))

43551

In [32]:
len(set(ep.get_all_qids()))

1252201